# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [ ]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd

def load_labeled_tsv(path):
    df = pd.read_csv(path, sep="\t", header=None)
    numeric = df.iloc[1:, 1:].astype(float).to_numpy()
    col_labels = df.iloc[0, 1:].tolist()
    row_labels = df.iloc[1:, 0].tolist()
    return numeric, col_labels, row_labels

# Load A, B, Q, R
A, A_cols, A_rows = load_labeled_tsv("model-A.tsv")
B, B_cols, B_rows = load_labeled_tsv("model-B.tsv")
Q, Q_cols, Q_rows = load_labeled_tsv("cost-Q.tsv")
R, R_cols, R_rows = load_labeled_tsv("cost-R.tsv")

# Basic shape checks
n = A.shape[0]
assert A.shape == (n, n), "A must be square"
assert B.shape[0] == n, "B must have same number of rows as A"
assert Q.shape == (n, n), "Q must be n×n"
assert R.shape[0] == B.shape[1] and R.shape[1] == B.shape[1], "R must be m×m where m = inputs"

# Solve the discrete-time Algebraic Riccati Equation (DARE)
# Use SciPy; otherwise fall back to simple fixed-point Riccati iteration.
try:
    from scipy.linalg import solve_discrete_are
    P = solve_discrete_are(A, B, Q, R)
except Exception:
    # Fixed-point iteration (sufficient here since (A,B) is stabilizable & (Q^(1/2),A) detectable)
    P = Q.copy()
    for _ in range(10000):
        BtPB = B.T @ P @ B
        K_tmp = np.linalg.inv(R + BtPB) @ (B.T @ P @ A)
        P_next = A.T @ P @ A - A.T @ P @ B @ np.linalg.inv(R + BtPB) @ B.T @ P @ A + Q
        if np.linalg.norm(P_next - P, ord='fro') < 1e-10:
            P = P_next
            break
        P = P_next

# LQR gain: K = (R + B^T P B)^{-1} B^T P A
K = np.linalg.inv(R + B.T @ P @ B) @ (B.T @ P @ A)

# Pretty output with labels (rows: inputs, cols: states)
K_df = pd.DataFrame(K, index=R_rows, columns=A_cols)
display(K_df)

,x,v,a
u,0.33446,1.304454,1.858131


Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [3]:
# YOUR CHANGES HERE

K_df.to_csv("control-K-intended.tsv", sep="\t")
print("Saved control-K-intended.tsv")

Saved control-K-intended.tsv


Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [4]:
# YOUR CHANGES HERE

import os
import numpy as np
import pandas as pd

def load_labeled_tsv(path):
    df = pd.read_csv(path, sep="\t", header=None)
    numeric = df.iloc[1:, 1:].astype(float).to_numpy()
    col_labels = df.iloc[0, 1:].tolist()
    row_labels = df.iloc[1:, 0].tolist()
    return numeric, col_labels, row_labels

def compute_K_from_files():
    # Fallback in case K.tsv isn't present
    A, A_cols, _ = load_labeled_tsv("model-A.tsv")
    B, _, _ = load_labeled_tsv("model-B.tsv")
    Q, _, _ = load_labeled_tsv("cost-Q.tsv")
    R, _, _ = load_labeled_tsv("cost-R.tsv")
    try:
        from scipy.linalg import solve_discrete_are
        P = solve_discrete_are(A, B, Q, R)
    except Exception:
        # Simple fixed-point Riccati iteration
        P = Q.copy()
        for _ in range(10000):
            BtPB = B.T @ P @ B
            inv_term = np.linalg.inv(R + BtPB)
            P_next = A.T @ P @ A - A.T @ P @ B @ inv_term @ B.T @ P @ A + Q
            if np.linalg.norm(P_next - P, ord='fro') < 1e-10:
                P = P_next
                break
            P = P_next
    K = np.linalg.inv(R + B.T @ P @ B) @ (B.T @ P @ A)
    return pd.DataFrame(K, index=["u"], columns=A_cols)

In [5]:
if os.path.exists("control-K-intended.tsv"):
    K_df = pd.read_csv("control-K-intended.tsv", sep="\t", index_col=0)
else:
    K_df = compute_K_from_files()
    K_df.to_csv("control-K-intended.tsv", sep="\t")

log = pd.read_csv("qc-log.tsv", sep="\t")
state_cols = list(K_df.columns)   # ensures we match K's column ordering (e.g., ["x","v","a"])
X = log[state_cols].to_numpy().T  # shape (n, T)
K = K_df.to_numpy()               # shape (m, n); here m = 1

u_check = (-K @ X).flatten()

Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [6]:
# YOUR CHANGES HERE

qc_check = pd.DataFrame({
    "index": log["index"].astype(int).values,
    "u_check": u_check
})
qc_check.to_csv("qc-check.tsv", sep="\t", index=False)
print('Saved "qc-check.tsv" with columns: index, u_check')
display(qc_check.head(10))

Saved "qc-check.tsv" with columns: index, u_check


,index,u_check
0,0,1.672299
1,1,-1.152095
2,2,-1.235183
3,3,-0.288504
4,4,0.369202
5,5,0.430598
6,6,0.188644
7,7,-0.024803
8,8,-0.089025
9,9,-0.052384


Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [ ]:
# YOUR CHANGES HERE

def load_labeled_tsv(path):
    df = pd.read_csv(path, sep="\t", header=None)
    numeric = df.iloc[1:, 1:].astype(float).to_numpy()
    col_labels = df.iloc[0, 1:].tolist()
    row_labels = df.iloc[1:, 0].tolist()
    return numeric, col_labels, row_labels



In [ ]:
intended_path = "control-K-intended.tsv"
if os.path.exists(intended_path):
    intended_df = pd.read_csv(intended_path, sep="\t", header=None)
    state_cols = intended_df.iloc[0, 1:].tolist()   # e.g., ["x","v","a"]
    row_labels = intended_df.iloc[1:, 0].tolist()   # e.g., ["u"]
else:
    _, state_cols, _ = load_labeled_tsv("model-A.tsv")
    row_labels = ["u"]

# Load the prototype log
log = pd.read_csv("qc-log.tsv", sep="\t")

# Design matrix (states) and target (actions)
X = log[state_cols].to_numpy()      # shape (N, n_states)
y = log["u"].to_numpy()             # shape (N,)

# Linear regression WITHOUT intercept to recover u = K_actual x
# This captures the control actually used by the prototype.
coef, *_ = np.linalg.lstsq(X, y, rcond=None)   # coef shape (n_states,)
K_actual = coef.reshape(1, -1)                  # shape (1, n_states)

K_actual_df = pd.DataFrame(K_actual, index=row_labels, columns=state_cols)

Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [9]:
# YOUR CHANGES HERE

out_df = pd.DataFrame([["index"] + state_cols])
for rname in row_labels:
    out_df = pd.concat([out_df, pd.DataFrame([[rname] + list(K_actual_df.loc[rname].values)])], ignore_index=True)

out_df.to_csv("control-K-actual.tsv", sep="\t", header=False, index=False)
print('Saved "control-K-actual.tsv"')
display(K_actual_df)

Saved "control-K-actual.tsv"


,x,v,a
u,-0.340438,-1.30012,-1.950117


Submit "control-k-actual.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [14]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd
import os

def load_labeled_tsv(path):
    df = pd.read_csv(path, sep="\t", header=None)
    numeric = df.iloc[1:, 1:].astype(float).to_numpy()
    col_labels = df.iloc[0, 1:].tolist()
    row_labels = df.iloc[1:, 0].tolist()
    return numeric, col_labels, row_labels

def save_labeled_matrix(path, M, row_labels, col_labels, float_fmt="%.12f"):
    cols = ["index"] + col_labels                   # shared column names
    header = pd.DataFrame([["index"] + col_labels], columns=cols)
    body = pd.DataFrame(M, columns=col_labels)
    body.insert(0, "index", row_labels)
    out = pd.concat([header, body], ignore_index=True)
    out.to_csv(path, sep="\t", header=False, index=False,
               float_format=float_fmt, lineterminator="\n", encoding="utf-8")

_, state_cols, state_rows = load_labeled_tsv("model-A.tsv")   # states (e.g., ['x','v','a'])
_, input_cols, _          = load_labeled_tsv("model-B.tsv")   # inputs (e.g., ['u'])


In [15]:
# Load the training log with noisy actions
train = pd.read_csv("qc-train.tsv", sep="\t")
assert all(c in train.columns for c in state_cols + input_cols), \
    f"qc-train.tsv must contain columns: {state_cols + input_cols}"

# Build regression: x_{t+1} = A x_t + B u_t + noise
X_t   = train[state_cols].to_numpy()[:-1, :]   # (T-1, n)
X_tp1 = train[state_cols].to_numpy()[1:, :]    # (T-1, n)
U_t   = train[input_cols].to_numpy()[:-1, :]   # (T-1, m)

Z = np.hstack([X_t, U_t])    # (T-1, n+m)
Y = X_tp1                    # (T-1, n)

# Solve least squares WITHOUT intercept: Y ≈ Z @ Theta
#    Theta stacks A and B column-wise: Theta = [A^T; B^T] with shape (n+m, n)
Theta, *_ = np.linalg.lstsq(Z, Y, rcond=None)

n, m = len(state_cols), len(input_cols)
A_new = Theta[:n, :].T       # (n, n)
B_new = Theta[n:, :].T       # (n, m)

Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [16]:
# YOUR CHANGES HERE

save_labeled_matrix("model-A-new.tsv", A_new, state_rows, state_cols)
save_labeled_matrix("model-B-new.tsv", B_new, state_rows, input_cols)
print('Saved "model-A-new.tsv" and "model-B-new.tsv"')

print(pd.read_csv("model-A-new.tsv", sep="\t", header=None).head().to_string(index=False))
print(pd.read_csv("model-B-new.tsv", sep="\t", header=None).head().to_string(index=False))

Saved "model-A-new.tsv" and "model-B-new.tsv"
    0                      1                     2                     3
index                      x                     v                     a
    x     1.0000000000000016    1.1000000000000032 2.886579864025407e-15
    v -1.281954983795886e-16    0.9000000000000001    0.9499999999999982
    a -9.313780314509612e-17 5.551115123125783e-16    1.0999999999999999
    0                       1
index                       u
    x -1.1102230246251565e-16
    v    -0.01000000000000062
    a                     0.9


Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.